In [82]:
import pandas as pd 
import plotly.express as px 
import warnings 
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")

In [83]:
stETH_ETH_apr_df = pd.read_csv(
    ROOT_DATA_DIR
    / 'daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv',
    index_col=0,
    parse_dates=True,
).dropna()

##### Reward rate data

In [84]:
stETH_ETH_incentive_rewards_df = pd.read_parquet(
    ROOT_DATA_DIR / 'raw_pool_data/curve_stETH_ETH/extra_rewards_df.parquet'
)
# quantity of LDO rewards issues per second 
px.line(stETH_ETH_incentive_rewards_df['reward_rate'])

### Individual behavior analysis of LARGEST WHALE

In [85]:
lp_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18
2021-01-05 13:36:06,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x182B723a58739a9c974cFDB385ceaDb237453c28,69.999492,Transfer,11594833,50,43,0xacd73e0a596914d36650b57a0bf264cc8a8b63e053a8...,2021-01-05 13:36:06
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47
...,...,...,...,...,...,...,...,...,...
2023-06-21 00:00:11,0x99ac10631F69C753DDb595D074422a0922D9056B,0x182B723a58739a9c974cFDB385ceaDb237453c28,0.003230,Transfer,17524329,349,316,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x182B723a58739a9c974cFDB385ceaDb237453c28,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0.003230,Transfer,17524329,349,318,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0xF403C135812408BFbE8713b5A23a04b3D48AAE31,0.003230,Transfer,17524329,349,321,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11


In [86]:
null_address = '0x0000000000000000000000000000000000000000'
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]

In [87]:
withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1
warnings.filterwarnings('ignore')
withdraw_liquidity_transfers['value']

timestamp
2021-01-05 17:12:33     -2.000945
2021-01-05 17:46:38   -499.501433
2021-01-05 21:03:54     -0.999754
2021-01-06 03:36:23    -10.037649
2021-01-06 12:17:18     -1.002055
                          ...    
2023-06-20 22:25:35     -0.000000
2023-06-20 22:33:11    -65.033669
2023-06-20 22:40:35    -51.167100
2023-06-20 23:28:11     -0.000000
2023-06-21 00:00:11     -0.003230
Name: value, Length: 27851, dtype: float64

In [88]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

warnings.filterwarnings('ignore')

In [109]:
# finding cumulative sum 
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()

# sorting from smallest to largest (by user's largest balance at one point in time)
whales_sorted_df = transfer_df.groupby('user').max().sort_values('cumsol_col')
whales_sorted_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202


In [108]:
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()

# classifying whales as those who have over 50,000 eth 
whale_df = transfer_df.loc[transfer_df['cumsol_col'] > 50000]


In [110]:
largest_whale = transfer_df[transfer_df['user'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
largest_whale

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2022-09-26 14:36:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389,Transfer,15618180,367,385,0x200853bc39a0b4ac57f3a396d22a39a070686ec5e63a...,2022-09-26 14:36:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389
2022-09-26 19:08:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0.921205,Transfer,15619528,39,152,0xe62463f53c97cc3542eb72887719022b33172209954b...,2022-09-26 19:08:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118783.220593
2022-09-27 04:24:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.909552,Transfer,15622291,162,246,0xfb072e072c6557451effe6a61158eed4360decd747c2...,2022-09-27 04:24:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118785.130146
2022-09-27 13:44:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.903760,Transfer,15625068,172,316,0x21ba8a9e96c6d327fb6df54ff59a683ed8dc418fa2c7...,2022-09-27 13:44:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118787.033906
2022-09-27 23:17:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.905536,Transfer,15627904,76,160,0xa1add66f1689c1ffc90c12e53dd7f4fde42a066ccea3...,2022-09-27 23:17:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118788.939442
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428863,87,247,0xaf7747639152a7093b205d38308052f6fba17716c65c...,2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,107303.607420
2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428867,163,204,0x7183029355114ef096e371caae8dc304e9c21f78cecc...,2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,105303.607420
2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428869,70,133,0xfea33c7d1cf4a21e4269a595047bd551dbd8af65b5da...,2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,103303.607420


#### How many transactions did they take part in? 

In [92]:
added = add_liquidity_transfers[add_liquidity_transfers['to'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
len(added)

185

In [93]:
withdrew = withdraw_liquidity_transfers[withdraw_liquidity_transfers['from'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
len(withdrew)

14

In [94]:
# number of transactions this whale took part in
num_transactions = len(added) - len(withdrew) 
num_transactions

171

In [95]:
# number of transctions for whales w/ 50k+ ETH
top_21_whales  = transfer_df.groupby('user').max().sort_values('cumsol_col').tail(21)
for i, whale in enumerate(top_21_whales.index):
    added = add_liquidity_transfers[add_liquidity_transfers['to'] == whale]
    withdrew = withdraw_liquidity_transfers[withdraw_liquidity_transfers['from'] == whale]
    num_transactions = len(added) - len(withdrew) 
    print(i + 1, ":", whale, num_transactions)

1 : 0x712d0f306956a6A4b4F9319aD9B9de48c5345996 -2
2 : 0x2339814f8B162b66087F9C12F7Ff268b7fD498Ea 4
3 : 0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A 702
4 : 0xD275E5cb559D6Dc236a5f8002A5f0b4c8e610701 4
5 : 0xF9fDc2B5F60355A237deb8BD62CC117b1C907f7b 16
6 : 0xcdE35b62c27d70b279cf7d0AA1212fFA9e938CEf 1
7 : 0xBc79855178842FDBA0c353494895DEEf509E26bB 3
8 : 0xca2C8b7664FA4169bd85DA72A968DaB9b78F5882 0
9 : 0x7cCD3bEfb83154B99C02F4DD5AeC5dD76f1ee0b2 0
10 : 0xA86e412109f77c45a3BC1c5870b880492Fb86A14 1
11 : 0xBC8d100e2c7C1d6BbEfc3128bd5185d226a1976A 3
12 : 0x47cC445c8845F7186A1eaB87Ae5D60CDA69b630c 2
13 : 0x6Cf9AA65EBaD7028536E353393630e2340ca6049 23
14 : 0x3BA21b6477F48273f41d241AA3722FFb9E07E247 10
15 : 0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE 1
16 : 0x25AD8b7cd0738331E65b92736b71A4979691E479 1
17 : 0x12b5c9191E186658841F2431943c47278f68E075 1
18 : 0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1 3
19 : 0xEAA7723633cf598E872D611f5EC50a45b65CBc72 4
20 : 0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3 171
2

##### How long did this whale stay in the pool? 

In [96]:
whales_sorted_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202


In [97]:
whales_sorted_df[whales_sorted_df['cumsol_col'] < 0]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0x0000000000000000000000000000000000000000,-0.000523,Transfer,17468383,101,210,0xc50f9a6f6c75f21eb03d618f5493992de4fd5ce65b89...,2023-06-13 03:22:59,-0.000523
0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x0000000000000000000000000000000000000000,-0.000009,Transfer,16911337,135,285,0xeeccbc0406c55d79980852f93aa84e221b9582fe4018...,2023-03-26 11:34:59,-0.000521
0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0x0000000000000000000000000000000000000000,-0.000100,Transfer,13898358,78,139,0xd9b0107373665e1166eebd3b95df6aaf98fe21bd93bd...,2021-12-29 05:34:32,-0.000100


##### There are 1,351 unique users who have a negative balance. We will consider them as not part of the pool anymore. 

In [98]:
# choose random user from this group of people who are not part of the pool anymore
person = '0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A'
add_liquidity_transfers[add_liquidity_transfers["to"] == person]


,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,


In [99]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2023-03-14 03:13:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000010,Transfer,16823446,53,161,0x358adf18c89be02cfa114e8a316b8d7159cf19429064...,2023-03-14 03:13:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 06:17:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000188,Transfer,16838583,111,276,0x440e9910c26cbd8920a05bd9afa6d79fceb65b97599c...,2023-03-16 06:17:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 06:49:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000188,Transfer,16838742,68,229,0x999da014b76f2c9238b9a8d4f145b2a2af9924c8f00a...,2023-03-16 06:49:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 09:07:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.001323,Transfer,16839420,74,185,0x9bbb863e89c591eed89ebc8867202b5ddc3a89874317...,2023-03-16 09:07:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-28 02:07:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.006572,Transfer,16922769,35,32,0xa2add4120a7a7685049c4800873d40f45fd55da0b9f6...,2023-03-28 02:07:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A


##### This doesn't make sense to me because this person ONLY withdrew from the pool and never added liquidity. I noticed this with multiple users with negative balances.

### Whale's balance over time compared to LDO reward rate 

In [142]:
px.bar(largest_whale['cumsol_col'])

In [141]:
# the actual graph of whale balance over time vs. reward rate 
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

# top 21 whales graphs (whales who had over 50,000 eth cumulatively)
top_21_whales  = transfer_df.groupby('user').max().sort_values('cumsol_col').tail(21)
# filtering for transactions after 2022
top_21_whales_filtered = top_21_whales[top_21_whales['timestamp'] > '2022-01-01']

for whale in top_21_whales_filtered.index:
    largest_whale = transfer_df[transfer_df['user'] == whale]
    print(whale)
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
    go.Bar(x=largest_whale.index, y=largest_whale['cumsol_col'].ffill(), name='balance over time'),
    secondary_y=False,
    )
    fig.add_trace(
    go.Scatter(x=stETH_ETH_incentive_rewards_df.index, y=stETH_ETH_incentive_rewards_df['reward_rate_eth'], name='reward rate (eth)'),
    secondary_y=True, 
    )

    fig.update_xaxes(title_text='timestamp')
    fig.update_yaxes(title_text='balance over time', secondary_y=False)
    fig.update_yaxes(title_text='reward rate (eth)', secondary_y=True)
    fig.show()

0x712d0f306956a6A4b4F9319aD9B9de48c5345996


0x2339814f8B162b66087F9C12F7Ff268b7fD498Ea


0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A


0xD275E5cb559D6Dc236a5f8002A5f0b4c8e610701


0xF9fDc2B5F60355A237deb8BD62CC117b1C907f7b


...

0x3BA21b6477F48273f41d241AA3722FFb9E07E247


0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE


0x12b5c9191E186658841F2431943c47278f68E075


0xEAA7723633cf598E872D611f5EC50a45b65CBc72


0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3


### As LP incentives are removed from a specific pool (by the DAO who is paying incentives), what is the (in)elasticity of the TVL?

In [102]:
# merge stETH_ETH_apr_df with incentive_rewards_df 
incentive_rewards_filtered = stETH_ETH_incentive_rewards_df.loc[stETH_ETH_incentive_rewards_df.index.isin(stETH_ETH_apr_df.index)]
merged_df = pd.merge_asof(stETH_ETH_apr_df, stETH_ETH_incentive_rewards_df['reward_rate'], left_index=True, right_index=True)


In [103]:
merged_df_2 = pd.merge_asof(merged_df, stETH_ETH_incentive_rewards_df['reward_rate_eth'], left_index=True, right_index=True)

In [101]:
# elasticity of tvl
stETH_ETH_apr_df['token_supply_shifted'] = stETH_ETH_apr_df['lp_token_total_supply'].shift(-30) 
stETH_ETH_apr_df['incentive_apr_shifted'] = stETH_ETH_apr_df['raw_incentive_apr'].shift(-30) 

# setting variables 
q2 = stETH_ETH_apr_df['lp_token_total_supply']
q1 = stETH_ETH_apr_df['token_supply_shifted']
p2 = stETH_ETH_apr_df['raw_incentive_apr'] # change to reward rate
p1 = stETH_ETH_apr_df['incentive_apr_shifted'] 

# elasticity formula 
change_in_supply = (p2 - p1) / ((p2 + p1) / 2) * 100
change_in_price = (q2 - q1) / ((q2 + q1) / 2) * 100
stETH_ETH_apr_df['elasticity'] = change_in_supply / change_in_price
stETH_ETH_apr_df['elasticity']


timestamp
2022-10-10   -0.236856
2022-10-11   -0.544266
2022-10-12   -0.615040
2022-10-13   -0.522298
2022-10-14   -0.510939
                ...   
2023-06-17         NaN
2023-06-18         NaN
2023-06-19         NaN
2023-06-20         NaN
2023-06-21         NaN
Name: elasticity, Length: 255, dtype: float64

Graph 

In [104]:
# scaled appropriately 
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(
go.Scatter(x=merged_df_2.index, y=merged_df_2['elasticity'], name='elasticity'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=merged_df_2.index, y=merged_df_2['reward_rate_eth'], name='reward_rate_eth'),
    secondary_y=True, 
    )

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='elasticity', secondary_y=False)
fig.update_yaxes(title_text='reward rate (eth)', secondary_y=True)

### Aggregate behavior of multiple whales

In [105]:
# top whales graphs
top_whales = transfer_df.groupby('user').max().sort_values('cumsol_col').tail(15).sum()
top_whales
largest_whale = transfer_df[transfer_df['user'] == top_whales.index[0][1]]
print(top_whales.index)
print(top_whales)
print(largest_whale)
# merging stETH ETH df onto largest whale df so we can access the reward rate and cumsol col in the same dataframe
reward_time_largest_whale = stETH_ETH_incentive_rewards_df.loc[stETH_ETH_incentive_rewards_df.index.isin(largest_whale['timestamp'].dt.normalize())]
reward_time_largest_whale = reward_time_largest_whale.sort_index()
    
merged_largest_whale = pd.merge_asof(largest_whale, reward_time_largest_whale['reward_rate_eth'], left_index=True, right_index=True)
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
go.Scatter(x=merged_largest_whale.index, y=merged_largest_whale['cumsol_col'], name='cumsol_col'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=merged_largest_whale.index, y=merged_largest_whale['reward_rate_eth'], name='reward_rate_eth'),
    secondary_y=True, 
    )

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='balance over time', secondary_y=False)
fig.update_yaxes(title_text='reward rate (eth)', secondary_y=True)
fig.show()

Index(['from', 'to', 'value', 'event', 'block', 'transaction_index',
       'log_index', 'hash', 'cumsol_col'],
      dtype='object')
from                 0xBc79855178842FDBA0c353494895DEEf509E26bB0xca...
to                   0xBc79855178842FDBA0c353494895DEEf509E26bB0xca...
value                                                   1046944.747079
event                TransferTransferTransferTransferTransferTransf...
block                                                        222092036
transaction_index                                                 3749
log_index                                                         6533
hash                 0xee95c3dadb3cb441a80cc51928eb302763fb50dfad6a...
cumsol_col                                              1438395.095872
dtype: object
Empty DataFrame
Columns: [from, to, value, event, block, transaction_index, log_index, hash, timestamp, user, cumsol_col]
Index: []
